In [1]:
import scanpy as sc
from pathlib import Path
from dask import array as da
from tqdm.dask import TqdmCallback

from scatlastb_utils.io import read_anndata, write_zarr_linked

/Users/michaela.mueller/miniforge3/envs/scanpy_test/lib/python3.11/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
/Users/michaela.mueller/miniforge3/envs/scanpy_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = 'Hrovatin2023'
file_path = f'data/{dataset_name}.h5ad'

# Get data

In [3]:
def get_cxg_url(collection_id, dataset_id):
    """
    Adapted code from https://github.com/chanzuckerberg/single-cell-curation/blob/main/notebooks/curation_api/python_raw/get_dataset.ipynb
    """
    import requests
    print(f'Get URL for CxG collection ID "{collection_id}" and dataset ID "{dataset_id}"')

    url = f'https://api.cellxgene.cziscience.com/curation/v1/collections/{collection_id}/datasets/{dataset_id}/'
    assets = requests.get(url=url).json()['assets']
    asset = [a for a in assets if a["filetype"] == "H5AD"][0]
    url = asset["url"]

    print(f'URL: {url}')
    return url

In [4]:
url = get_cxg_url(
    collection_id="296237e2-393d-4e31-b590-b03f74ac5070",
    dataset_id="49e4ffcc-5444-406d-bdee-577127404ba8"
)

Get URL for CxG collection ID "296237e2-393d-4e31-b590-b03f74ac5070" and dataset ID "49e4ffcc-5444-406d-bdee-577127404ba8"
URL: https://datasets.cellxgene.cziscience.com/49243c50-bf0c-4b10-87f8-55ec9f455399.h5ad


In [ ]:
# download only if the file does not exist
!curl -C - -o {file_path} {url}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 47 3884M   47 1834M    0     0  22.5M      0  0:02:51  0:01:21  0:01:30 26.7M

In [ ]:
adata = read_anndata(
    file_path,
    X='raw/X',
    obs='obs',
    var='var',
    obsm='obsm',
    obsp='obsp',
    dask=True,
    backed=True,
)
adata

## Visualise data

In [ ]:
sc.pl.embedding(
    adata,
    'X_integrated_umap',
    color=['cell_type', 'cell_type_reannotatedIntegrated', 'dataset'],
    ncols=1,
    frameon=False,
)

# Preprocess data

In [ ]:
adata.layers['counts'] = adata.X.copy()

In [ ]:
if isinstance(adata.X, da.Array):
    with TqdmCallback(leave=False):
        adata.X = adata.X.compute()

In [ ]:
sc.pp.normalize_total(adata)

In [ ]:
sc.pp.log1p(adata)

In [ ]:
%%time
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key='dataset')

In [ ]:
%%time
sc.pp.pca(adata, n_comps=50, mask_var='highly_variable')

In [ ]:
%%time
sc.pp.neighbors(adata, use_rep='X_pca')

In [ ]:
%%time
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(
    adata,
    color=[
        'cell_type',
        'cell_type_reannotatedIntegrated',
        'dataset',
    ],
    ncols=1,
    frameon=False,
)

In [ ]:
adata.write_zarr(f'data/{dataset_name}_preprocessed.zarr')